# **CIROH DocuHub - Database Population and Content Embedding**

In [1]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
import time
import openai
import json

In [2]:
# Environment variables
load_dotenv()

PG_HOST = os.getenv("POSTGRES_HOST")
PG_DB = os.getenv("POSTGRES_DB")
PG_USER = os.getenv("POSTGRES_USER")
PG_PASS = os.getenv("POSTGRES_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")

BASE_URL = "https://docs.ciroh.org"


In [3]:
# Databes connection
conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DB,
    user=PG_USER,
    password=PG_PASS
)

def execute_query(conn, query, params=None, fetch=False):
    cur = conn.cursor()
    """Execute a SQL query with optional parameters."""
    try:
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        if fetch:
            result = cur.fetchall()
            return result
        else:
            conn.commit()
    except Exception as e:
        print(f"Error executing query: {e}")
        conn.rollback()
        return None
    finally:
        cur.close()


In [4]:
# Initialize OpenAI client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

## **DocuHub website**

In [ ]:
# URL of the HTML page to scrape the site structure
HTML_URL = "https://docs.ciroh.org/docs/products/intro"

# Download and parse the HTML content
response = requests.get(HTML_URL)
response.raise_for_status()  # Raise an error if the download fails
soup = BeautifulSoup(response.text, "html.parser")

# Top-bar navigation links
navbar = soup.find("nav", {"aria-label": "Main"})

# Find the left and right blocks
navbar_left = navbar.find("div", class_="theme-layout-navbar-left")
navbar_right = navbar.find("div", class_="theme-layout-navbar-right")

top_level_links = []

# 1. Explicit capture of DocuHub
docuhub_link = navbar_left.find("a", class_="navbar__brand")
if docuhub_link:
    name = docuhub_link.get_text(strip=True)
    href = docuhub_link.get("href")
    top_level_links.append({"name": name, "url": href})

# 2. Left block (except CIROH Portal)
for a in navbar_left.find_all("a", class_="navbar__item"):
    name = a.get_text(strip=True)
    href = a.get("href")
    if "portal" in name.lower():
        continue  # Omit CIROH Portal
    top_level_links.append({"name": name, "url": href})

# 3. Right block
for a in navbar_right.find_all("a", class_="navbar__item"):
    name = a.get_text(strip=True)
    href = a.get("href")
    top_level_links.append({"name": name, "url": href})

# Show results
for link in top_level_links:
    print(f'Name: {link["name"]:20} | URL: {link["url"]}')

In [28]:
# Insert Content Types into the table TBLContentTypes
content_types = [{
    "idtype": 1,
    "typename": "Text"
},
{
    "idtype": 2,
    "typename": "Table"
}]

for row in content_types:
    # Check if the content type already exists
    existing = execute_query(conn, """
        SELECT * FROM TBLContentTypes WHERE idtype = %s
    """, (row['idtype'],))
    if not existing:
        execute_query(conn, """
            INSERT INTO TBLContentTypes (idtype, typename)
            VALUES (%s, %s)
        """, (row['idtype'], row['typename']))

Error executing query: duplicate key value violates unique constraint "tblcontenttypes_pkey"
DETAIL:  Key (idtype)=(1) already exists.

Error executing query: duplicate key value violates unique constraint "tblcontenttypes_pkey"
DETAIL:  Key (idtype)=(2) already exists.



In [ ]:
# Insert links into the database
for link in top_level_links:
    execute_query(conn, """
        INSERT INTO tblurls (url, idurlparent, name)
        VALUES (%s, NULL, %s)
        ON CONFLICT DO NOTHING
    """, (link["url"], link["name"]))
    print(f'Inserted: {link["name"]} | {link["url"]}')


In [ ]:
# Manually insert a page into the database that is not in the top-level links
execute_query(conn, """
    INSERT INTO tblurls (url, idurlparent, name)
    VALUES (%s, (SELECT idurl FROM tblurls WHERE name = %s), %s)
""", ("/docs/contribute", "DocuHub", "Contributing to CIROH DocuHub"))


In [ ]:
# Extract URLs from the left sidebar menu
def extract_menu_links(soup):
    """
    Extracts all visible links in the left sidebar menu of the current page.
    """
    results = []
    main_ul = soup.find("ul", class_="theme-doc-sidebar-menu menu__list")
    if not main_ul:
        return results
    for a_tag in main_ul.find_all("a", class_="menu__link"):
        url = a_tag.get("href")
        name = a_tag.get_text(strip=True)
        if url:
            results.append({"url": url, "name": name})
    return results

def extract_menu(start_url, verbose=False):
    """
    Recursively extracts the full left sidebar menu structure from a given starting URL.
    Returns a list of dictionaries with: url, urlparent, name.
    """
    queue = [start_url]
    extracted = []
    visited = set()

    while queue:
        current_url = queue.pop(0)
        if current_url in visited:
            continue
        visited.add(current_url)
        page_url = BASE_URL + current_url
        if verbose:
            print(f"Processing: {page_url}")
        try:
            resp = requests.get(page_url)
            if resp.status_code != 200:
                continue
            soup = BeautifulSoup(resp.text, "html.parser")
            links = extract_menu_links(soup)
            for link in links:
                # Only add if this URL is not already in 'extracted'
                if not any(x['url'] == link['url'] for x in extracted):
                    extracted.append({
                        "url": link['url'],
                        "urlparent": current_url,
                        "name": link['name']
                    })
                    if link['url'] not in visited and link['url'] not in queue:
                        queue.append(link['url'])
            time.sleep(0.1)
        except Exception as e:
            print(f"Error with {page_url}: {e}")    
    return extracted


In [ ]:
# Insert link from left sidebar menu into the database
urls_to_insert = execute_query(conn, """
    SELECT url FROM tblurls 
    WHERE name in ('Products', 'Services', 'Policies', 'Contributing to CIROH DocuHub') 
    ORDER BY idurl
""", fetch=True)


for url in urls_to_insert:
    url = url[0]  # unpack tuple
    print(f"Extracting menu for {url}...")
    df = pd.DataFrame(extract_menu(url, verbose=False))
    for _, row in df.iterrows():
        # Check if the URL already exists in the database
        exists = execute_query(conn, """
            SELECT 1 FROM tblurls WHERE url = %s
        """, (row['url'],), fetch=True)
        if not exists:
            # Insert the new URL into the database
            execute_query(conn, """
                INSERT INTO tblurls (url, idurlparent, name)
                VALUES (%s, (SELECT idurl FROM tblurls WHERE url = %s), %s)
            """, (row['url'], row['urlparent'], row['name']))
            print(f'Inserted: {row["name"]} | {row["url"]}')


In [ ]:
# Extract blog links from the sidebar
def extract_blog_links(start_url="/blog"):
    """
    Extracts all blog post URLs and titles from the blog sidebar.
    Returns a list of dicts with url, urlparent, and name.
    """
    results = []
    page_url = BASE_URL + start_url
    resp = requests.get(page_url)
    if resp.status_code != 200:
        print(f"Failed to load {page_url}")
        return results
    
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # Find the sidebar container
    sidebar = soup.find("nav", class_="sidebar_brwN")
    if not sidebar:
        print("Sidebar not found.")
        return results
    
    # Loop through each year group
    for group in sidebar.find_all("div", role="group"):
        
        ul = group.find("ul", class_="sidebarItemList_QwSx")
        print(f"Found ul: {ul}")
        if not ul:
            continue
        
        for a_tag in ul.find_all("a", class_="sidebarItemLink_yNGZ"):
            url = a_tag.get("href")
            name = a_tag.get_text(strip=True)
            if url:
                results.append({
                    "url": url,
                    "urlparent": start_url,
                    "name": name
                })
                print(f'Found blog link: {name} | {url}')
    
    return results

In [ ]:
# Insert blog links into the database
blog_links = extract_blog_links("/blog")

for link in blog_links:
    exists = execute_query(conn, """
        SELECT 1 FROM tblurls WHERE url = %s
    """, (link['url'],), fetch=True)
    if not exists:
        # Insert the new URL into the database
        execute_query(conn, """
            INSERT INTO tblurls (url, idurlparent, name)
            VALUES (%s, (SELECT idurl FROM tblurls WHERE url = %s), %s)
        """, (link['url'], link['urlparent'], link['name']))
        print(f'Inserted: {link["name"]} | {link["url"]}')


In [11]:
# Prompt for generating summaries

def build_prompt(markdown_content):
    """
    Builds the prompt for the language model by inserting the provided markdown content.
    """

    prompt_template = f"""
# ROLE & GOAL
You are an AI Knowledge Architect specializing in hydrology and scientific computing for the Cooperative Institute for Research to Operations in Hydrology (CIROH). Your task is to process the content of a webpage from CIROH's DocuHub documentation site.

# CONTEXT
The output you generate will be stored in a PostgreSQL database to power a Retrieval-Augmented Generation (RAG) system. Specifically, your response will populate a `JSONB` field. The `summary_text` within your JSON response will be used to create a 3072-dimension vector embedding for semantic search. Therefore, the summary must be information-dense, conceptually accurate, and optimized to be found by user queries about CIROH's products, services, policies, and research activities.

# CONTENT TO ANALYZE
---
**[BEGINNING OF MARKDOWN CONTENT]**

{markdown_content}

**[END OF MARKDOWN CONTENT]**
---

# INSTRUCTIONS
Based on the content provided above, generate a single JSON object. Follow these rules precisely:

1.  **Focus on "What" and "Why"**: The `summary_text` should clearly identify the page's core purpose. Is it a product description, a tutorial, a policy document, a release note, or a blog post?
2.  **Extract Named Entities**: The `entities` array must include all specific names of products, services, tools, technologies, standards, or key collaborators mentioned (e.g., "NextGen in A Box", "Research Datastream", "Hydrofabric", "NOAA", "BYU Hydroinformatics Courses").
3.  **Capture Key Concepts**: The `keywords` array should capture the main technical or thematic concepts, not just proper nouns (e.g., "hydrologic modeling", "flood inundation mapping", "water quality", "data management", "cloud computing", "machine learning").
4.  **Synthesize, Don't Quote**: Do not copy sentences directly from the text. Synthesize the information in your own words to create a coherent and semantically rich summary.
5.  **Be Dense and Direct**: Start the summary directly with the subject. Avoid introductory phrases. For example, instead of writing "This page describes NextGen In A Box...", you must write "NextGen In A Box is a community-accessible...". Be direct and objective.

# REQUIRED OUTPUT FORMAT
Generate a single, valid JSON object with the following structure. Do not add any text or explanation before or after the JSON object.

{{
  "summary_text": "A single, dense paragraph summarizing the content according to the instructions.",
  "keywords": ["An", "array", "of", "string", "keywords"],
  "entities": ["An", "array", "of", "string", "entities"],
  "document_type": "A single string classifying the document (e.g., 'Product Description', 'Tutorial', 'Policy', 'Release Note', 'Blog Post')"
}}
"""
    return prompt_template

In [12]:
# Generate summary from markdown content

SCRAPED_PATH = "../CIROH-scraper/docs_ciroh_org"

def generate_summary(url_markdown):

    def load_markdown_content(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                return file.read().strip()
        except FileNotFoundError:
            return None
        
    # Match URL with Scraped content
    processed_path = url_markdown.lstrip('/')
    processed_path = processed_path.removeprefix("docs/")
    processed_path = processed_path.replace("release-notes", "release_notes")
    processed_path = processed_path.replace(" ", "%20")

    full_path = os.path.join(SCRAPED_PATH, processed_path, "index.md")

    markdown_content = load_markdown_content(full_path)

    if markdown_content is None:
        print(f"Markdown content not found at: {full_path}")
        return None

    try:
        response = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "system", "content": "You are an AI Knowledge Architect specializing in hydrology and scientific computing for the Cooperative Institute for Research to Operations in Hydrology (CIROH)."},
                {"role": "user", "content": build_prompt(markdown_content)},
            ],
            response_format={"type": "json_object"},
            # temperature=0.01
            reasoning_effort = "minimal",
            verbosity = "low"
        )

        summary_json = json.loads(response.choices[0].message.content.strip())
        print(f"Generated summary for {url_markdown}")
        return summary_json
    except Exception as e:
        print(f"Error occurred during summary generation for {url_markdown}: {e}")



In [14]:
urls_to_summarize = execute_query(conn, """
    SELECT idurl, url FROM tblurls WHERE summary_data IS NULL ORDER BY idurl;
""", fetch=True)

for idurl, url in urls_to_summarize:
    summary_json = generate_summary(url)
    if summary_json is not None:
        # Insert the summary into the database
        execute_query(conn, """
            UPDATE tblurls
            SET summary_data = %s
            WHERE idurl = %s
        """, (json.dumps(summary_json), idurl))
        print(f'Updated summary for idurl: {idurl}')

Generated summary for /docs/products/data-management/dataaccess/NWMURL Library
Updated summary for idurl: 74


In [ ]:
def get_breadcrumb(conn, url_id):
    """
    Generates the breadcrumb trail for a given idurl using a recursive query.

    Args:
        conn: The database connection object.
        url_id: The idurl of the page for which to generate the trail.

    Returns:
        A string with the breadcrumb (e.g., "Home > Products > NGIAB"),
        or None if an error occurs or the idurl is not found.
    """

    query = """
    WITH RECURSIVE breadcrumb_path AS (
        -- Anchor Member: Select the starting page
        SELECT
            idurl,
            name,
            idurlparent,
            1 AS depth -- Initial depth level
        FROM
            tblurls
        WHERE
            idurl = %s

        UNION ALL

        -- Recursive Member: Join the table to find the parent
        SELECT
            u.idurl,
            u.name,
            u.idurlparent,
            bp.depth + 1 -- Increment depth at each level
        FROM
            tblurls u
        JOIN
            breadcrumb_path bp ON u.idurl = bp.idurlparent
    )
    -- Select the final result, aggregating the names into a single string
    SELECT
        string_agg(name, ' > ' ORDER BY depth DESC) AS breadcrumb
    FROM
        breadcrumb_path;
    """
    
    try:
        result = execute_query(conn, query, params=(url_id,), fetch=True)
        
        if result and result[0] and result[0][0]:
            return result[0][0]
        else:
            return None

    except Exception as e:
        print(f"An error occurred in get_breadcrumb for idurl {url_id}: {e}")
        return None

In [ ]:
breadcrumb_trail = get_breadcrumb(conn, 73)
print(breadcrumb_trail)

In [12]:
def get_embedding(text, dimensions, model=EMBEDDING_MODEL):
    """Call OpenAI to get an embedding for the given text."""
    try:
        response = openai.embeddings.create(
            input=text,
            model=model,
            dimensions=dimensions
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"❌ Error generating embedding: {e}")
        return None

In [17]:
# Generate embeddings for the summary text

'''
urls_to_embed = execute_query(conn, """
    SELECT idurl, name || ':\n\n' || (summary_data ->> 'summary_text') summary
    FROM tblurls
    WHERE summary_data IS NOT NULL AND embedding IS NULL
    ORDER BY idurl
""", fetch=True)
'''
urls_to_embed = execute_query(conn, """
    SELECT idurl, summary_data ->> 'summary_text' summary
    FROM tblurls
    WHERE summary_data IS NOT NULL 
    ORDER BY idurl
""", fetch=True)

for idurl, summary in urls_to_embed:
    embeddings = get_embedding(summary, 1792)
    if embeddings is not None:
        # Insert the summary into the database
        execute_query(conn, """
            UPDATE tblurls
            SET embedding = %s
            WHERE idurl = %s
        """, (embeddings, idurl))
        print(f'Updated embedding for idurl: {idurl}')

Updated embedding for idurl: 1
Updated embedding for idurl: 2
Updated embedding for idurl: 3
Updated embedding for idurl: 4
Updated embedding for idurl: 5
Updated embedding for idurl: 6
Updated embedding for idurl: 7
Updated embedding for idurl: 8
Updated embedding for idurl: 9
Updated embedding for idurl: 10
Updated embedding for idurl: 11
Updated embedding for idurl: 12
Updated embedding for idurl: 13
Updated embedding for idurl: 14
Updated embedding for idurl: 15
Updated embedding for idurl: 16
Updated embedding for idurl: 17
Updated embedding for idurl: 18
Updated embedding for idurl: 19
Updated embedding for idurl: 20
Updated embedding for idurl: 21
Updated embedding for idurl: 22
Updated embedding for idurl: 23
Updated embedding for idurl: 24
Updated embedding for idurl: 25
Updated embedding for idurl: 26
Updated embedding for idurl: 27
Updated embedding for idurl: 28
Updated embedding for idurl: 29
Updated embedding for idurl: 30
Updated embedding for idurl: 31
Updated embedding

In [14]:
def populate_chunks_from_json(json_file_path):
    """
    Loads content chunks from a JSON file and inserts them into the TBLContent table.
    """
    # 1. Load the chunks from the JSON file into a Pandas DataFrame
    try:
        df_chunks = pd.read_json(json_file_path)
        print(f"✅ JSON file '{json_file_path}' loaded successfully. Found {len(df_chunks)} chunks.")
    except FileNotFoundError:
        print(f"❌ ERROR: The file '{json_file_path}' was not found.")
        return
    except Exception as e:
        print(f"❌ ERROR: Failed to read JSON file. Details: {e}")
        return

    # 2. Create a mapping from TypeName to idType from the database
    print("\n--- Fetching content type mappings from database ---")
    type_rows = execute_query(conn, "SELECT idType, TypeName FROM TBLContentTypes;", fetch=True)
    if not type_rows:
        print("❌ ERROR: Could not fetch content types from TBLContentTypes. Please ensure the table is populated.")
        return
    
    type_to_id_map = {name.lower(): type_id for type_id, name in type_rows}
    print(f"✔️ Mappings created: {type_to_id_map}")

    # 3. Iterate over the DataFrame and insert each chunk into TBLContent
    print("\n--- Populating TBLContent with new chunks ---")
    insert_count = 0
    for index, row in df_chunks.iterrows():
        idurl = row['idurl']
        content_type_name = row['idtype']
        order = row['order']
        content = row['content']
        
        # Look up the foreign key for idType
        idtype_fk = type_to_id_map.get(content_type_name.lower())

        if idtype_fk is None:
            print(f"⚠️ WARNING: Content type '{content_type_name}' not found in TBLContentTypes. Skipping row {index}.")
            continue

        # Insert the chunk into the database
        # Using ON CONFLICT to prevent errors if you run the script on the same file twice
        execute_query(conn, """
            INSERT INTO TBLContent (idURL, idType, "order", Content)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (idurl, idtype_fk, order, content))
        insert_count += 1

        if (index + 1) % 100 == 0:
            print(f"Processed {index + 1} of {len(df_chunks)} chunks...")

    print(f"\n✅ Process complete. {insert_count} new chunks were processed for insertion.")


In [20]:
# Populate TBLContent from the JSON file
populate_chunks_from_json("documentation_chunks_v2.json")

✅ JSON file 'documentation_chunks_v2.json' loaded successfully. Found 440 chunks.

--- Fetching content type mappings from database ---
✔️ Mappings created: {'text': 1, 'table': 2}

--- Populating TBLContent with new chunks ---
Processed 100 of 440 chunks...
Processed 200 of 440 chunks...
Processed 300 of 440 chunks...
Processed 400 of 440 chunks...

✅ Process complete. 440 new chunks were processed for insertion.


In [16]:
# Generate embeddings for the content chunks
chunks_to_embed = execute_query(conn, """
    SELECT idcontent, content FROM TBLContent
    WHERE content IS NOT NULL AND embedding IS NULL
    ORDER BY idcontent
""", fetch=True)

for idcontent, content in chunks_to_embed:
    embeddings = get_embedding(content, 1792)
    if embeddings is not None:
        # Insert the summary into the database
        execute_query(conn, """
            UPDATE TBLContent
            SET embedding = %s
            WHERE idcontent = %s
        """, (embeddings, idcontent))
        print(f'Updated embedding for idcontent: {idcontent}')



Updated embedding for idcontent: 1405
Updated embedding for idcontent: 1406
Updated embedding for idcontent: 1407
Updated embedding for idcontent: 1408
Updated embedding for idcontent: 1409
Updated embedding for idcontent: 1410
Updated embedding for idcontent: 1411
Updated embedding for idcontent: 1412
Updated embedding for idcontent: 1413
Updated embedding for idcontent: 1414
Updated embedding for idcontent: 1415
Updated embedding for idcontent: 1416
Updated embedding for idcontent: 1417
Updated embedding for idcontent: 1418
Updated embedding for idcontent: 1419
Updated embedding for idcontent: 1420
Updated embedding for idcontent: 1421
Updated embedding for idcontent: 1422
Updated embedding for idcontent: 1423
Updated embedding for idcontent: 1424
Updated embedding for idcontent: 1425
Updated embedding for idcontent: 1426
Updated embedding for idcontent: 1427
Updated embedding for idcontent: 1428
Updated embedding for idcontent: 1429
Updated embedding for idcontent: 1430
Updated embe

## **HydroShare CIROH Community**

In [6]:
# HydroShare links

# Load HydroShare links from CSV
df_hydroshare = pd.read_csv("hydroshare_links.csv", sep=';')

# Insert links into the database
for link in df_hydroshare.to_dict(orient='records'):
    execute_query(conn, """
        INSERT INTO tblurls (url, idurlparent, name)
        VALUES (%s, NULL, %s)
        ON CONFLICT DO NOTHING
    """, (link["url"], link["name"]))
    print(f'Inserted: {link["name"]} | {link["url"]}')


Inserted: HydroShare is CUAHSI's online collaboration environment for sharing data, models, and code | https://www.hydroshare.org/
Inserted: HydroShare Support | https://help.hydroshare.org/
Inserted: HydroShare Discover | https://www.hydroshare.org/search/
Inserted: Cooperative Institute for Research to Operations in Hydrology (CIROH) | https://www.hydroshare.org/community/4/
Inserted: High Water Marks July 2024 Vermont Floods - UVM Floodplains | https://www.hydroshare.org/resource/4d98edaf289942e49ba1e2491d123a32/
Inserted: Enabling collaboration through data and model sharing with CUAHSI HydroShare; CIROH Devcon 25 presentation | https://www.hydroshare.org/resource/023bfa101586432ba0f6ed9ddfea60a9/
Inserted: Enabling collaboration through data and model sharing with CUAHSI HydroShare; CIROH Science Meeting Poster | https://www.hydroshare.org/resource/75e311b73ba64f259ec52668e458ca99/
Inserted: Clear Creek at Golden, Colorado (USGS 06719505): Nextgen v2.2 hydrofabric and AORC forcing

In [5]:
# Update HydroShare links with its parent idurl
execute_query(conn, """
    UPDATE tblurls
    SET idurlparent = (SELECT idurl FROM tblurls WHERE url = %s)
    WHERE url IN ('https://help.hydroshare.org/', 'https://www.hydroshare.org/search/', 'https://www.hydroshare.org/community/4/') AND idurlparent IS NULL
""", ("https://www.hydroshare.org/", ))

execute_query(conn, """
    UPDATE tblurls
    SET idurlparent = (SELECT idurl FROM tblurls WHERE url = %s)
    WHERE idurl > (SELECT idurl FROM tblurls WHERE url = %s) AND idurlparent IS NULL
""", ("https://www.hydroshare.org/community/4/", "https://www.hydroshare.org/"))

In [7]:
def build_hydroshare_prompt(page_content):
    """
    Builds the prompt for the language model by inserting the provided HydroShare page content.
    """
    prompt_template = f"""
# ROLE & GOAL
You are an AI Knowledge Architect specializing in hydrology and scientific data management. Your task is to process the content of a resource-sharing page from the HydroShare platform.

# CONTEXT
The output you generate will be stored in a PostgreSQL database to power a Retrieval-Augmented Generation (RAG) system for the Cooperative Institute for Research to Operations in Hydrology (CIROH). The `summary_text` will be used to create a vector embedding for semantic search. Therefore, the summary must be information-dense, conceptually accurate, and optimized for user queries about hydrological datasets, models, and tools.

# CONTENT TO ANALYZE
---
**[BEGINNING OF WEBPAGE CONTENT]**

{page_content}

**[END OF WEBPAGE CONTENT]**
---

# INSTRUCTIONS
Based on the content provided above, generate a single JSON object. Follow these rules precisely:

1.  **Focus on "What" and "Why"**: The `summary_text` should clearly identify the resource's core purpose. Is it a dataset, a computational notebook, a model, a collection of files? What scientific question does it address or what problem does it solve?
2.  **Extract Named Entities**: The `entities` array must include specific names of datasets, models, tools, geographic locations, contributing organizations, or key authors mentioned (e.g., "National Water Model", "Jupyter Notebook", "Logan River, Utah", "CUAHSI", "USGS").
3.  **Capture Key Concepts**: The `keywords` array should capture the main technical or thematic concepts (e.g., "hydrologic modeling", "time series data", "geospatial analysis", "water quality", "data sharing", "reproducible science").
4.  **Synthesize, Don't Quote**: Do not copy sentences directly. Synthesize the information to create a coherent and semantically rich summary.
5.  **Be Dense and Direct**: Start the summary directly with the subject. For example, instead of "This resource contains a dataset...", write "Dataset of streamflow measurements for the...". Be direct and objective.

# REQUIRED OUTPUT FORMAT
Generate a single, valid JSON object with the following structure. Do not add any text or explanation before or after the JSON object.

{{
  "summary_text": "A single, dense paragraph summarizing the content according to the instructions.",
  "keywords": ["An", "array", "of", "string", "keywords"],
  "entities": ["An", "array", "of", "string", "entities"],
  "document_type": "A single string classifying the resource (e.g., 'Dataset', 'Computational Notebook', 'Model', 'Collection', 'Geographic Feature')"
}}
"""
    return prompt_template

In [ ]:
def fetch_and_clean_content(url):
    """
    Downloads and cleans the main content from a HydroShare page.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()  # Raise an error for 4xx or 5xx responses

        # BeautifulSoup to parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        if soup.body:
            return soup.body.get_text(separator=' ', strip=True)
        return None

    except requests.exceptions.RequestException as e:
        print(f"Error al descargar la URL {url}: {e}")
        return None

In [ ]:
def generate_summary_from_url(url):
    """
    Executes the full process: fetches content, cleans it, and generates a summary for a given URL.
    """
    print(f"Processing URL: {url}")
    page_content = fetch_and_clean_content(url)

    if not page_content:
        print("Could not retrieve content to process.")
        return None

    try:
        response = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "system", "content": "You are an AI Knowledge Architect specializing in hydrology and scientific data management."},
                {"role": "user", "content": build_hydroshare_prompt(page_content)},
            ],
            response_format={"type": "json_object"},
            reasoning_effort = "minimal",
            verbosity = "low"
        )

        summary_json = json.loads(response.choices[0].message.content.strip())
        print(f"Summary generated for {url}")
        return summary_json
    except Exception as e:
        print(f"Error during summary generation for {url}: {e}")
        return None

In [ ]:
# 1. Get HydroShare URLs without summaries
urls_to_summarize = execute_query(conn, """
    SELECT idurl, url FROM tblurls 
    WHERE url LIKE '%%hydroshare.org%%' AND summary_data IS NULL 
    ORDER BY idurl;
""", fetch=True)

# 2. Iterate and generate summaries
for idurl, url in urls_to_summarize:
    summary_json = generate_summary_from_url(url)
    if summary_json:
        # Insert the summary into the database
        execute_query(conn, """
            UPDATE tblurls
            SET summary_data = %s
            WHERE idurl = %s;
        """, (json.dumps(summary_json), idurl))
        print(f'--> Summary updated in the database for idurl: {idurl}\n')
    else:
        print(f'--> Summary generation failed for idurl: {idurl}\n')
    
    # Pause to avoid overloading HydroShare servers
    time.sleep(2)

print("Process completed.")


In [15]:
# Populate TBLContent from the JSON file
populate_chunks_from_json("hydroshare_links_chunks.json")

✅ JSON file 'hydroshare_links_chunks.json' loaded successfully. Found 498 chunks.

--- Fetching content type mappings from database ---
✔️ Mappings created: {'text': 1, 'table': 2}

--- Populating TBLContent with new chunks ---
Processed 100 of 498 chunks...
Processed 200 of 498 chunks...
Processed 300 of 498 chunks...
Processed 400 of 498 chunks...

✅ Process complete. 498 new chunks were processed for insertion.


In [9]:
# Close the database connection
conn.close()